<a href="https://colab.research.google.com/github/binhvd/Data-Management-2/blob/main/PySpark/NYC_Parking_Tickets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download files from Google Cloud Storage

In [ ]:
!wget https://storage.googleapis.com/dm2-data/new-york/Parking_Violations_Issued_-_Fiscal_Year_2015.csv
!wget https://storage.googleapis.com/dm2-data/new-york/Parking_Violations_Issued_-_Fiscal_Year_2016.csv
!wget https://storage.googleapis.com/dm2-data/new-york/Parking_Violations_Issued_-_Fiscal_Year_2017.csv

## Try to import records to DataFrames

In [ ]:
import glob
import pandas as pd   
import time

data = []
start_time = time.time()

for path in glob.glob("/content/*.csv"):
  df = pd.read_csv(path)
  data.append(df)

df = pd.concat(data)
print("--- %s seconds ---" % (time.time() - start_time))

## Connect to Cluster

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("yarn")\
        .appName('NYC-Parking-Tickets')\
        .getOrCreate()

## Read CSV files to DataFrame


In [ ]:
data = spark.read.csv(
    'gs://dm2-data/new-york/*.csv',
    sep = ',',
    header = True,
)

data.printSchema()

## Count records

In [ ]:
data.count()

## Count which cars get the most tickets

In [ ]:
from pyspark.sql import functions as f
import time

start_time = time.time()
data.groupBy("Vehicle Make").count().sort(f.col("count").desc()).show()
print("--- %s seconds ---" % (time.time() - start_time))